In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": '',
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/7706ecae-ce08-4ff4-bd4d-110c2bb741c2/oauth2/token"}

dbutils.fs.mount(
source = "abfss://container@storageacc.dfs.core.windows.net", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)




In [ ]:
%fs
ls "/mnt/tokyoolymic"

In [ ]:
spark

In [ ]:
atheletics = spark.read.format("csv").option("header","true").load("/mnt/tokyoolymic/raw_data/atheletics.csv")
coaches = spark.read.format("csv").option("header","true").load("/mnt/tokyoolymic/raw_data/coaches.csv")
entriesGender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw_data/EntriesGender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw_data/Medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw_data/Teams.csv")
atheletics.show()

In [ ]:
atheletics.printSchema

In [ ]:
coaches.show()

In [ ]:
entriesGender.show()

In [ ]:
entriesGender.printSchema

In [ ]:
medals.show()

In [ ]:
teams.show()

In [ ]:
# Starting transformation

In [ ]:
entriesGender= entriesGender.withColumn("Female",col("Female").cast(IntegerType()))
entriesGender= entriesGender.withColumn("Male",col("Male").cast(IntegerType()))
entriesGender= entriesGender.withColumn("Total",col("Total").cast(IntegerType()))
entriesGender.printSchema

In [ ]:
medals.printSchema

In [ ]:
# Find the countries with top gold medals
top_gold_medal_countries= medals.orderBy("gold",ascending=False).select("Team_Country","gold")
top_gold_medal_countries.show()

In [ ]:
# calculate the number of enteries by gender for each discipline
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesGender.withColumn(
    'Avg_Female', entriesGender['Female'] / entriesGender['Total']
).withColumn(
    'Avg_Male', entriesGender['Male'] / entriesGender['Total']
)
average_entries_by_gender.show()

In [ ]:
atheletics.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed_data/atheletics")
entriesGender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed_data/entriesGender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed_data/medals")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed_data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed_data/teams")